In [0]:
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": "app_registration_client_id",
    "fs.azure.account.oauth2.client.secret": 'app_registration_secret_id',
    "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/app_registration_tenant_id/oauth2/token"
}

dbutils.fs.mount(
    source = "abfss://olympicdata-tokyo@dataengineeringfp.dfs.core.windows.net", # container_name@storage_account_name
    mount_point = "/mnt/olympicdata",
    extra_configs = configs
)

True

In [0]:
%fs
ls "/mnt/olympicdata"

path,name,size,modificationTime
dbfs:/mnt/olympicdata/raw-data/,raw-data/,0,1714610937000
dbfs:/mnt/olympicdata/transformed-data/,transformed-data/,0,1714610946000


In [0]:
spark

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [0]:
# .option("inferSchema", "true") Assigns Proper data types to the columns in table

In [0]:
athletes = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/olympicdata/raw-data/athletes.csv")
coaches = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/olympicdata/raw-data/coaches.csv")
entriesgender    = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/olympicdata/raw-data/entriesgender.csv")
medals = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/olympicdata/raw-data/medals.csv")
teams = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/olympicdata/raw-data/teams.csv")

In [0]:
athletes.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [0]:
athletes.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [0]:
coaches.show()

+--------------------+--------------------+-----------------+--------+
|                Name|             Country|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    NULL|
|           ABE Junya|               Japan|       Volleyball|    NULL|
|       ABE Katsuhiko|               Japan|       Basketball|    NULL|
|        ADAMA Cherif|       C�te d'Ivoire|         Football|    NULL|
|          AGEBA Yuya|               Japan|       Volleyball|    NULL|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
|ALLER CARBALLO Ma...|               Spain|       Basketball|    NULL|
|     

In [0]:
coaches.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
entriesgender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
entriesgender = entriesgender.withColumn(colName="Female", col=col("Female").cast(IntegerType())) \
    .withColumn(colName="Male", col=col("Male").cast(IntegerType())) \
    .withColumn(colName="Total", col=col("Total").cast(IntegerType()))


In [0]:
entriesgender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
medals.show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|        Team_Country|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [0]:
medals.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Team_Country: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [0]:
# Find the top countries with the highest number of gold medals
medals.orderBy("Gold", ascending=False).show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|        Team_Country|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  14|                Cuba|   7|     3|     5|   15|           18|
|  13|         New Zealand|   7|     6|     7|   20|          

In [0]:
medals.orderBy("Gold", ascending=False).select("Team_Country", "Gold").show()

+--------------------+----+
|        Team_Country|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
|           Australia|  17|
|         Netherlands|  10|
|              France|  10|
|             Germany|  10|
|               Italy|  10|
|                Cuba|   7|
|         New Zealand|   7|
|              Brazil|   7|
|              Canada|   7|
|             Hungary|   6|
|   Republic of Korea|   6|
|               Kenya|   4|
|              Poland|   4|
|      Czech Republic|   4|
|              Norway|   4|
+--------------------+----+
only showing top 20 rows



In [0]:
# Calculate the average number of entries by gender for each discipline
entriesgender.withColumn('Avg_Male', entriesgender['Male'] / entriesgender['Total']).withColumn('Avg_Female', entriesgender['Female'] / entriesgender['Total']).show()

+--------------------+------+----+-----+-------------------+-------------------+
|          Discipline|Female|Male|Total|           Avg_Male|         Avg_Female|
+--------------------+------+----+-----+-------------------+-------------------+
|      3x3 Basketball|    32|  32|   64|                0.5|                0.5|
|             Archery|    64|  64|  128|                0.5|                0.5|
| Artistic Gymnastics|    98|  98|  196|                0.5|                0.5|
|   Artistic Swimming|   105|   0|  105|                0.0|                1.0|
|           Athletics|   969|1072| 2041| 0.5252327290543851| 0.4747672709456149|
|           Badminton|    86|  87|  173| 0.5028901734104047|0.49710982658959535|
|   Baseball/Softball|    90| 144|  234| 0.6153846153846154|0.38461538461538464|
|          Basketball|   144| 144|  288|                0.5|                0.5|
|    Beach Volleyball|    48|  48|   96|                0.5|                0.5|
|              Boxing|   102

In [0]:
# Calculate the average number of entries by gender for each discipline
entriesgender.withColumn('Avg_Male', entriesgender['Male'] / entriesgender['Total']).withColumn('Avg_Female', entriesgender['Female'] / entriesgender['Total']).select("Discipline", "Avg_Male", "Avg_Female").show()

+--------------------+-------------------+-------------------+
|          Discipline|           Avg_Male|         Avg_Female|
+--------------------+-------------------+-------------------+
|      3x3 Basketball|                0.5|                0.5|
|             Archery|                0.5|                0.5|
| Artistic Gymnastics|                0.5|                0.5|
|   Artistic Swimming|                0.0|                1.0|
|           Athletics| 0.5252327290543851| 0.4747672709456149|
|           Badminton| 0.5028901734104047|0.49710982658959535|
|   Baseball/Softball| 0.6153846153846154|0.38461538461538464|
|          Basketball|                0.5|                0.5|
|    Beach Volleyball|                0.5|                0.5|
|              Boxing| 0.6470588235294118|0.35294117647058826|
|        Canoe Slalom|                0.5|                0.5|
|        Canoe Sprint| 0.5060240963855421| 0.4939759036144578|
|Cycling BMX Frees...|0.47368421052631576| 0.5263157894

In [0]:
athletes.write.option("header", "true").csv("/mnt/olympicdata/transformed-data/athletes") # To write one time only

In [0]:
athletes.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/olympicdata/transformed-data/athletes")
medals.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/olympicdata/transformed-data/medals")
entriesgender.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/olympicdata/transformed-data/entriesgender")
coaches.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/olympicdata/transformed-data/coaches")
teams.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/olympicdata/transformed-data/teams")